In [1]:
%load_ext autoreload
%autoreload 2

from src.util import get_path
get_path()

include_pre_2021_data = 0

C:/Users/jy/AppData/Local/bipy/31094803/projects/7bacdd446cf1406ca946bab3a4997531


In [2]:
try:
    import bql
    import sys

    sys.path.append('./Lib')
    from etiqalib import *
    from etiqalib.config import *
    from etiqalib.bqfields import BaseGetData
    from datetime import datetime

    read = False

    bq = bql.Service()
    def px_function(bq, ticker, start, end):
        bqapi = BaseGetData(bq)
        dff = bqapi.get_data(ticker,{'px_last':bq.data.px_last(start=start, end=end, **fillprev)})
        return dff.drop('CURRENCY',1).dropna(subset=['px_last'])

    start_date = '1998-01-01'
    end_date = datetime.today().date().isoformat()

    if not read:
        opr_df = download_incremental(bq, 'MAOPRATE Index',px_function, start_date, end_date)
        srr_df = download_incremental(bq, 'MAOPSRR Index',px_function, start_date, end_date)
        m3_df = download_incremental(bq, 'MAMS3YOY Index',px_function, start_date, end_date)
        magy10yr = download_incremental(bq, 'MAGY10YR Index',px_function, start_date, end_date)
        price_df = download_incremental(bq, 'FBMKLCI Index', px_function, start_date, end_date)

        opr_df.to_csv('data/opr/opr_rate.csv')
        srr_df.to_csv('data/opr/srr_rate.csv')
        m3_df.to_csv('data/opr/m3_rate.csv')
        magy10yr.to_csv('data/opr/magy10yr.csv')
        price_df.to_csv('data/opr/fbmklci.csv')
        dt = opr_df["DATE"].max()
        print(f"Refreshed data: {dt}")
    else:
        opr_df = pd.read_csv('data/opr/opr_rate.csv', parse_dates=['DATE'])
        srr_df = pd.read_csv('data/opr/srr_rate.csv', parse_dates=['DATE'])
        m3_df = pd.read_csv('data/opr/m3_rate.csv', parse_dates=['DATE'])
        magy10yr = pd.read_csv('data/opr/magy10yr.csv', parse_dates=['DATE'])
        price_df = pd.read_csv('data/opr/fbmklci.csv', parse_dates=['DATE'])
except Exception as err:
    print("Running on local.")
    

Running on local.


In [3]:
raise_1 = False
get_path()

try:
    import bql
    raise_1 = True
except:
    raise_1 = False
if raise_1:
    raise Exception("Query only in BQNT environment.")

C:/Users/jy/AppData/Local/bipy/31094803/projects/7bacdd446cf1406ca946bab3a4997531


In [4]:
from src.jinja_html import render_html, encode_graph
import weasyprint
import pandas as pd
from datetime import timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', 50) 

import os
from src_new.data_loader import read_csv

# Read Data

In [5]:
from src_new.data_loader import Data
from src_new.chart_npl_1 import create_overview_fig, get_npl_text
from src_new.chart_total_loans_2 import create_total_loans_growth_fig, get_loans_growth_text
from src_new.chart_total_appl_appr_3 import create_loan_app_fig, get_total_applr_text
from src_new.chart_purposes_4 import create_fig_application, create_fig_approval
from src_new.table_purposes_6 import preproc_app_df, create_styled_table
from src_new.chart_4purposes_7 import create_4_purposes_figs, create_4_purposes_texts

In [6]:
data = Data()
data.query_bnm()

In [7]:
data.subset_date(dt="2024-05-31", loan_dt="2024-04-30")
# data.subset_date(dt="2024-04-30", loan_dt="2024-03-31")

# Chart 1 - NPL

In [8]:
m3_price_m = data.m3_price_m
srr_m = data.srr_m
npl = data.npl
opr_df = data.opr_df
fig_overview = create_overview_fig(m3_price_m, srr_m, npl, opr_df)
overview_text, npl_2, npl_1 = get_npl_text(m3_price_m, npl)
overview_text

'M3 YoY Growth is 6.06%. Non-performing Loan (NPL) is 1.63%, which is lower compared to last year (1.74%)'

# Chart 2 - Total Loans Growth

In [9]:
total_loans_df = data.total_loans_df
fig_loans_growth = create_total_loans_growth_fig(total_loans_df)
loans_growth_text = get_loans_growth_text(total_loans_df)
fig_loans_growth
loans_growth_text

'Total Loans Growth (YoY) is 6.05%; MoM Growth is 0.08%.'

# Chart 3 - Total Bank Loan Application and Approval

In [10]:
application_df = data.application_df
approval_df = data.approval_df
opr_df = data.opr_df
fig_loan = create_loan_app_fig(application_df, approval_df, opr_df) 
loan_text = get_total_applr_text(application_df, approval_df)
# fig_loan
loan_text

'As of April 2024, Total Bank Loan Application MoM and YoY change are 0.01% and 13.98% respectively; Total Bank Loan Approval MoM and YoY change are -2.09% and 12.84% respectively.'

# Chart 4, 5 - Loan App & Approval

In [11]:
application_df_2 = data.application_df_2
approval_df_2 = data.approval_df_2
fig_application = create_fig_application(application_df_2)
fig_approval = create_fig_approval(approval_df_2)
# display(fig_application)
# display(fig_approval)


# Table 6a, 6b - Loan App & Approval

In [12]:
appl_df_1 = preproc_app_df(application_df_2)
appr_df_1 = preproc_app_df(approval_df_2)
appl_df_2 = create_styled_table(appl_df_1)
appr_df_2 = create_styled_table(appr_df_1)
# display(appl_df_2)
# display(appr_df_2)

# Chart 7-10 - 4 Purposes 

In [13]:
purpose_figs = create_4_purposes_figs(application_df_2, approval_df_2)
purpose_texts = create_4_purposes_texts(appl_df_1, appr_df_1)

In [14]:
# for fig_, txt in zip(figs, purpose_texts):
#     display(fig_)
#     print(txt)
    

### Commentary

In [15]:

latest_month = application_df_2.index[-1].strftime("%B %Y")
lg_yoy = total_loans_df['Total_Loans'].pct_change(12).values[-1]
lg_mom = total_loans_df['Total_Loans'].pct_change(1).values[-1]

x1 = application_df.tail(13)
appl_1, appl_2, appl_3 = x1.loc[:, "TOTAL"].values[0], x1.loc[:, "TOTAL"].values[-2], x1.loc[:, "TOTAL"].values[-1]
appl_yoy = appl_3/appl_1-1
appl_mom = appl_3/appl_2-1

x1 = approval_df.tail(13)
appr_1, appr_2, appr_3 = x1.loc[:, "TOTAL"].values[0], x1.loc[:, "TOTAL"].values[-2], x1.loc[:, "TOTAL"].values[-1]
appr_yoy = appr_3/appr_1-1
appr_mom = appr_3/appr_2-1


In [16]:

commentary = f"""
*Banking Loan Stats Update (Data as of end of {latest_month}):*

*Total Loans*
Total loans YoY and MoM growth are *{lg_yoy:.2%}* and *{lg_mom:.2%}*, respectively.

*Loan Application and Approval*

Loan application and approval *MoM* growth are *{appl_mom:.2%}* and *{appr_mom:.2%}*, respectively. 
While, loan application and approval *YoY* growth are *{appl_yoy:.2%}* and *{appr_yoy:.2%}*, respectively.

Notable sectors in the credit market:
1) Credit Card (YoY growth - application: *{appl_df_1.loc['Credit Card', :][-1]:.2f}%*, approval: *{appr_df_1.loc['Credit Card', :][-1]:.2f}%*)
2) Passenger Cars (YoY growth - application: *{appl_df_1.loc['Passenger Cars', :][-1]:.2f}%*, approval: *{appr_df_1.loc['Passenger Cars', :][-1]:.2f}%*)
3) Working Capital (YoY growth - application: *{appl_df_1.loc['Working Capital', :][-1]:.2f}%*, approval: *{appr_df_1.loc['Working Capital', :][-1]:.2f}%*)
4) Residential Mortgages (YoY growth - application: *{appl_df_1.loc['Residential Mortgages', :][-1]:.2f}%*, approval: *{appr_df_1.loc['Residential Mortgages', :][-1]:.2f}%*)
5) Construction (YoY growth - application: *{appl_df_1.loc['Construction', :][-1]:.2f}%*, approval: *{appr_df_1.loc['Construction', :][-1]:.2f}%*)
"""

from IPython.display import display, Markdown, Latex
print(commentary)



*Banking Loan Stats Update (Data as of end of April 2024):*

*Total Loans*
Total loans YoY and MoM growth are *6.05%* and *0.08%*, respectively.

*Loan Application and Approval*

Loan application and approval *MoM* growth are *0.01%* and *-2.09%*, respectively. 
While, loan application and approval *YoY* growth are *13.98%* and *12.84%*, respectively.

Notable sectors in the credit market:
1) Credit Card (YoY growth - application: *-10.15%*, approval: *7.21%*)
2) Passenger Cars (YoY growth - application: *14.29%*, approval: *10.17%*)
3) Working Capital (YoY growth - application: *18.44%*, approval: *12.72%*)
4) Residential Mortgages (YoY growth - application: *14.83%*, approval: *11.89%*)
5) Construction (YoY growth - application: *33.85%*, approval: *23.44%*)



### Generate PDF

In [17]:
pd.to_datetime(latest_month)+pd.tseries.offsets.MonthEnd(1)

Timestamp('2024-04-30 00:00:00')

In [18]:
%%time
app_cols = ["Passenger Cars", "Residential Mortgages", "Credit Card", "Working Capital"]
dt = pd.to_datetime(latest_month)

date_filename = (pd.to_datetime(latest_month)+pd.tseries.offsets.MonthEnd(1)).strftime("%Y-%m-%d")
date_time = dt.strftime("%B %Y")

title = "Bank Loan Stats"
to_html_data = {
    "title": title,
    "date_time_str": f"{date_time} Update",
    
    "fig_overview": encode_graph(fig_overview),
    "overview_text": overview_text,
    
    "fig_loans_growth": encode_graph(fig_loans_growth),
    "loans_growth_text": loans_growth_text,
    
    "fig_loan": encode_graph(fig_loan),
    "loan_text": loan_text,
    "fig_application": encode_graph(fig_application),
    "fig_approval": encode_graph(fig_approval),
    "appl_df_2": appl_df_2.to_html(),
    "appr_df_2": appr_df_2.to_html(),
    "fig_titles": app_cols,
    "figs": [encode_graph(fig) for fig in purpose_figs],
    "purpose_texts": purpose_texts, 
    "figs_N": len(purpose_figs),
}

output_html_filename = f"output/{title}-{date_filename}.html"
output_pdf_filename = f"output/{title}-{date_filename}.pdf"
render_html(to_html_data, output_filename=output_html_filename,
            input_template_filename="output/bank_loan_stats_template.html")
print('Rendered HTML.')
os.add_dll_directory(r"C:\Program Files\GTK3-Runtime Win64\bin")
doc_1 = weasyprint.HTML(filename=output_html_filename)
doc_pdf = doc_1.write_pdf(output_pdf_filename)
print('Rendered PDF.')


Rendered HTML.
Rendered PDF.
CPU times: total: 375 ms
Wall time: 2.25 s


In [19]:
print("Done output:", output_pdf_filename)

Done output: output/Bank Loan Stats-2024-04-30.pdf
